In [ ]:
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Project/')

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import librosa
import IPython.display as ipd
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
# Get the base directory
basedir = os.getcwd()
dirname = basedir+ "/gtzan/audio_original"

# Create lists for audio paths and labels
audio_paths = []
audio_label = []
# Print all the files in different directories
for root, dirs, files in os.walk(dirname, topdown=False):
    for filenames in files:
        if filenames.find('.wav') != -1:

            audio_paths.append(os.path.join(root, filenames))
            filenames = filenames.split('.', 1)
            filenames = filenames[0]
            audio_label.append(filenames)
audio_paths = np.array(audio_paths)
audio_label = np.array(audio_label)
# audio_paths.shape

In [ ]:
# Create empty arrays for the features
AllSpec = np.empty([1000, 1025, 1293])
AllMel = np.empty([1000, 128, 1293])
AllMfcc = np.empty([1000, 10, 1293])

count = 0
# Create a list for the corrupt indices
bad_index = []
for i in tqdm(range(len(audio_paths))):
    try:

        path = audio_paths[i]
        y, sr = librosa.load(path)
        # For Spectrogram
        X = librosa.stft(y)
        Xdb = librosa.amplitude_to_db(abs(X))
        AllSpec[i] = Xdb

        # Mel-Spectrogram
        M = librosa.feature.melspectrogram(y=y)
        M_db = librosa.power_to_db(M)
        AllMel[i] = M_db

        # MFCC
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 10)
        AllMfcc[i] = mfcc

    except Exception as e:
        bad_index.append(i)

100%|██████████| 999/999 [15:35<00:00,  1.07it/s]


In [ ]:
import tensorflow

In [ ]:
# Delete the features at the corrupt indices
AllSpec = np.delete(AllSpec, bad_index, 0)
AllMel = np.delete(AllMel, bad_index, 0)
AllMfcc = np.delete(AllMfcc, bad_index, 0)


# Convert features to float32
AllSpec = AllSpec.astype(np.float32)
AllMel = AllMel.astype(np.float32)
AllMfcc = AllMfcc.astype(np.float32)


# Delete audio labels at corrupt indices
audio_label = np.delete(audio_label, bad_index)

# Convert labels from string into numerical value
audio_label[audio_label == 'blues'] = 0
audio_label[audio_label == 'classical'] = 1
audio_label[audio_label == 'country'] = 2
audio_label[audio_label == 'disco'] = 3
audio_label[audio_label == 'hiphop'] = 4
audio_label[audio_label == 'jazz'] = 5
audio_label[audio_label == 'metal'] = 6
audio_label[audio_label == 'pop'] = 7
audio_label[audio_label == 'reggae'] = 8
audio_label[audio_label == 'rock'] = 9
audio_label = [int(i) for i in audio_label]
audio_label = np.array(audio_label)

# Convert numerical data into categorical data
y = tensorflow.keras.utils.to_categorical(audio_label,num_classes = 10, dtype ="int32")

# Save the features and labels as a .npz file
np.savez_compressed(os.getcwd()+"/MusicFeatures.npz", spec= AllSpec, mel= AllMel, mfcc= AllMfcc, target=y)